In [ ]:
!pip install facenet-pytorch


In [ ]:
!git clone https://github.com/TachibanaYoshino/AnimeGANv2
%cd AnimeGANv2
!pip install -r requirements.txt


Cloning into 'AnimeGANv2'...
remote: Enumerating objects: 1735, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 1735 (delta 8), reused 8 (delta 0), pack-reused 1716 (from 1)
Receiving objects: 100% (1735/1735), 1013.15 MiB | 27.50 MiB/s, done.
Resolving deltas: 100% (141/141), done.
Updating files: 100% (322/322), done.
/content/AnimeGANv2
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
import gradio as gr
import torch
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np

# โหลดโมเดล AnimeGANv2 (สไตล์ face_paint_512_v2)
anime_model = torch.hub.load('bryandlee/animegan2-pytorch', 'generator', pretrained='face_paint_512_v2')
anime_model.eval()

# ฟังก์ชัน sharpen ภาพ
def sharpen(img: Image.Image) -> Image.Image:
    img_np = np.array(img)
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    sharpened = cv2.filter2D(img_np, -1, kernel)
    return Image.fromarray(sharpened)

# ฟังก์ชันแปลงภาพทั้งภาพเป็นอนิเมะ
def to_anime_full(img: Image.Image) -> Image.Image:
    original_size = img.size
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output_tensor = anime_model(input_tensor)[0]
    output_tensor = (output_tensor * 0.5 + 0.5).clamp(0, 1)
    output_img = transforms.ToPILImage()(output_tensor)
    return sharpen(output_img.resize(original_size))

# ฟังก์ชันแปลงภาพเป็นสเก็ต
def to_sketch(img: Image.Image) -> Image.Image:
    img_gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    inv_img = 255 - img_gray
    blur_img = cv2.GaussianBlur(inv_img, (21, 21), sigmaX=0, sigmaY=0)
    sketch = cv2.divide(img_gray, 255 - blur_img, scale=256)
    return Image.fromarray(sketch)

# สร้าง UI ด้วย Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🎨 ระบบ AI แปลงภาพจริงเป็นภาพอนิเมะและสเก็ต (ทั้งภาพ)")
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(type="pil", label="📤 อัปโหลดภาพจริง")
            btn_anime = gr.Button("✨ แปลงเป็นภาพอนิเมะ (ทั้งภาพ)")
            btn_sketch = gr.Button("🖊️ แปลงเป็นภาพสเก็ต")
        with gr.Column():
            output_img = gr.Image(label="📥 ผลลัพธ์ที่แปลงแล้ว")

    btn_anime.click(fn=to_anime_full, inputs=input_img, outputs=output_img)
    btn_sketch.click(fn=to_sketch, inputs=input_img, outputs=output_img)

demo.launch()


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/bryandlee/animegan2-pytorch/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bryandlee/animegan2-pytorch/raw/main/weights/face_paint_512_v2.pt" to /root/.cache/torch/hub/checkpoints/face_paint_512_v2.pt


100%|██████████| 8.20M/8.20M [00:00<00:00, 101MB/s]


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9009d01213036aef7b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
